## **SETUP**

In [1]:
# = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = 
# SOME COMMON OPTIONS AND LIBRARIES THAT I USE  = = = = = = = = = = = = = 
# = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = =

import urllib.request
# pandas para manipulação de dataframes
import pandas as pd

# = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = 

# upload de arquivos 
from google.colab import files

# = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = =
# Configurando o número máximo de linhas a se mostrar
pd.set_option('display.max_row', 5000)

# Configurando o número máximo de colunas a se mostrar
pd.set_option('display.max_columns', 50)

# Aumentando o número de caracteres a serem exibidos numa coluna de texto
pd.options.display.max_colwidth = 500

# Desligando a notificação setcopywarning
pd.set_option('mode.chained_assignment', None)

!pip install -U -q PyDrive
import os
import datetime as DT
import io
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import drive
from oauth2client.client import GoogleCredentials
import googleapiclient.discovery
import numpy as np 
import requests
import json
import time
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen 
from bs4 import BeautifulSoup 
import re

from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# **RETRIEVE ALL LINKS FOR ATHLETICS IN SUMMER OLYMPICS PAGE**

In [ ]:
###########################################################################################################
###
########## RETRIEVE ALL LINKS FOR "ATHLETICS IN SUMMER OLYMPICS" ON WIKIPEDIA
###
###########################################################################################################

res = requests.get("https://en.wikipedia.org/wiki/Athletics_at_the_Summer_Olympics")
soup = bs(res.text, "html.parser")
links = soup.select('a')
type(links)

athletic_links = []
for link in links:
    url = link.get("href", "")
    if "/wiki/Athletics" in url:

        d = {'url': url}         
        ooo = pd.DataFrame(d, index=[0])

        athletic_links.append(ooo) 

df_athletics = pd.concat(athletic_links).reset_index()
df_athletics

# NOW IT'S TIME TO FILTER AND RETAIN ONLY THOSE LINKS REGARDING OLYMPIC EDITIONS

df_athletics['link']     = df_athletics['url'].str.extract(r'(/wiki/Athletics_at_the_\d\d\d\d_Summer_Olympics_)*')
df_athletics2 = df_athletics.dropna(subset=['link']).reset_index()
df_athletics2 = df_athletics2[['url']]
df_athletics2.head(20)

# SOME REGEX CLEANNING

import re
df_athletics2['year']=df_athletics2['url'].str.extract(r'(\d{4})')
df_athletics2['year'] = df_athletics2['year'].astype(int)
df_athletics2.sort_values(['year'], ascending=[True], inplace=True)
df_athletics2 = df_athletics2.drop_duplicates()
#df_athletics2['type'] = df_athletics2['url'].str.extract(r'(?<=27s_).*')
df_athletics2['event'] = df_athletics2['url'].str.extract(r'(27s_[0-9A-za-z%,]+|Mixed_[0-9A-za-z%,]+).*')
df_athletics2['event'] = df_athletics2['event'].str.replace("27s_","")
df_athletics2['event'] = df_athletics2['event'].str.replace("%C3%97","x")
df_athletics2['event'] = df_athletics2['event'].str.replace("_"," ")
df_athletics2['men_women'] = df_athletics2['url'].str.extract(r'(Women|Men)') 

# DATAFRAME LINKS FOR EVERY EVENT FROM 1896 TO 2021, MAN AND WOMAN
df_athletics2.head(5)

A LOOK AT THE 100M

In [ ]:
df_athletics3 = df_athletics2.query('event == "100 metres"')
df_athletics3

# **BEAUTIFUL SOUP**

In [ ]:
import time

olympic_games = df_athletics2.query('event == "100 metres"')
olympic_games

sleep_min = 1
sleep_max = 3
start_time = time.time()

table_bodies  = []
heats_column0 = []
all_tables0 = []
headers = []
table_titles = []

for olympics in olympic_games.itertuples():
        
    url0 = getattr(olympics, 'url') 
    url  = 'https://en.wikipedia.org/'+url0
    #print(url)
    
    year = getattr(olympics, 'year')
    print(year)
    men_women = getattr(olympics, 'men_women')    
    event = getattr(olympics, 'event')

    page = urllib.request.urlopen(url)
    soup = BeautifulSoup(page, "html.parser")

    time.sleep(np.random.uniform(sleep_min, sleep_max))

    # ===========================================================================================================
    # 1 - TAKE EVERYTHING THAT'S BETWEEN "RESULTS" AND "FINAL" TABLE. THAT REPRESENTS WHAT WE WANT FROM THE PAGE
    # ===========================================================================================================

    heats = soup.find_all('span', {'class': 'mw-headline'})
    for i in heats:
      t = i.find_previous()
      h = t.find('span', {'class': 'mw-headline'}).text
      d = {'table_name': h, 'year': year, 'men_women': men_women}         
      ooo = pd.DataFrame(d, index=[0])
  
      table_titles.append(ooo)
      
    table_titles_final = pd.concat(table_titles).reset_index().drop(['index'], axis=1).reset_index()
    table_titles_final.rename(columns={"index": "id"}, inplace=True)
     
    table_titles_final['min_val'] = np.where( ((table_titles_final['table_name'] == "Results") | (table_titles_final['table_name'] == "Heats")), 1, np.nan)
    table_titles_final['max_val'] = np.where( ((table_titles_final['table_name'] == "Final")), 2,np.nan)
    table_titles_final['val'] = table_titles_final['min_val'].combine_first(table_titles_final['max_val']).fillna(0)#.astype(np.int64)
    table_titles_final = table_titles_final.drop(['min_val','max_val'], axis=1)
    table_titles_final

    # ==================================================================================
    # 2 - TAKE EVERY TABLE HEADER FROM ALL TABLES WITH "MW-HEADLINE"
    # ==================================================================================

    table_data = soup.find_all('table', {"class":["wikitable sortable", "wikitable"]})
    #print(len(table_data))

    # FOR EVERY TABLE IN MY TABLE DATA, I WANT THE "TH" TAG, IE., THE HEADER
    for i in table_data:
        th = i.find_all('th')

        # THEN I RETRIEVE THE TEXT FROM THE PREVIOUS SPAN TAG WITH MW-HEADLINE CLASS

        table_name = i.find_previous('span', {'class': 'mw-headline'}).text
        #print(table_name)
        count = 0
        
        for j in th:
            if len(j.text.strip()) <= 1:              
              th2 = "apagar"
              #print("vazio", len(j), j)
            else:
              th2 = j.text.strip()
              count = count+1
              #print(len(j), j)
            d = {'table_name': table_name, 'column_order_id': count, 'column_name': th2, 'year': year, 'men_women': men_women}         
            ooo = pd.DataFrame(d, index=[0])
            headers.append(ooo)

headers_final = pd.concat(headers).reset_index().drop(['index'], axis=1).reset_index()

# DELETING "OVERALL" HEADERS
headers_final = headers_final[ headers_final['table_name'].str.find("Overall") == -1 ]

# CLEANING TABLE NAMES THAT CONTAINS THE DOT "."
headers_final['table_name'] = headers_final['table_name'].str.replace(".","")
headers_final['column_name'] = headers_final['column_name'].str.replace(".","")
headers_final['column_name'] = headers_final['column_name'].str.replace("Country","Nation")
headers_final

table_titles_final['concat_for'] = table_titles_final['year'].astype(str) + table_titles_final['men_women']
table_titles_final

year_genre_loop = table_titles_final['concat_for'].unique().T
year_genre_loop = pd.DataFrame(year_genre_loop)
year_genre_loop.rename(columns={0: "year_genre"}, inplace=True)
year_genre_loop

right_tables = []

for ygenre in year_genre_loop.itertuples():
    
    filter = getattr(ygenre, 'year_genre')

    table_titles_final2 = table_titles_final.query('concat_for == @filter ')
    print(table_titles_final2)

    year = getattr(table_titles_final2, 'year')
    men_women = getattr(table_titles_final2, 'men_women')    
    val = getattr(table_titles_final2, 'val')    
    id = getattr(table_titles_final2, 'id')
    table_name = getattr(table_titles_final2, 'table_name')    

    if len(table_titles_final2.query('table_name=="Results"')) > 0:
      start = "Results"
    else:
      start = "Heats"

    df_min = table_titles_final2.query('table_name==@start')
    min_value = df_min['id'].values[0]
    min_value

    df_max = table_titles_final2.query('table_name=="Final"')
    max_value = df_max['id'].values[0]
    max_value

    table_titles_final3 = table_titles_final2.query('id >= @min_value & id<= @max_value ')
    right_tables.append(table_titles_final3)

tables_to_extract = pd.concat(right_tables)
tables_to_extract

table_scope0 = pd.merge(tables_to_extract, headers_final, left_on=['table_name', 'year', 'men_women'], right_on=['table_name', 'year', 'men_women'], how='inner').drop(['id', 'index'], axis=1)
table_scope = table_scope0[table_scope0.column_name != "apagar"]
table_scope.sort_index()
table_scope

table_scope = table_scope.drop(['val','concat_for'], axis=1)
table_scope1 = table_scope[['year']].drop_duplicates()

table_append=[]

for row in table_scope1.itertuples():
    #print(row)
    year = getattr(row, 'year')

    filtered = table_scope1.query('year == @year')

    for row2 in filtered.itertuples():
      filtered_year = table_scope[['table_name','year','men_women']].query('year == @year')
      unique_combination = filtered_year.drop_duplicates()
      
      filtered_genre = unique_combination[['men_women']].drop_duplicates()

      for row3 in filtered_genre.itertuples():

        men_women = getattr(row3, 'men_women')

        genre = unique_combination.query('men_women == @men_women')

        #print(unique_combination)
        genre['id'] = 1
        genre['idx'] = genre.id.cumsum()
        table_append.append(genre)

append_final = pd.concat(table_append)
append_final

'''
table_scope.to_csv('/content/drive/My Drive/headers_final.csv')
table_titles_final2.to_csv('/content/drive/My Drive/table_titles.csv')
'''


# **PANDAS HTML**

In [ ]:
olympic_games = df_athletics2.query('event == "100 metres"')
olympic_games

sleep_min = 1
sleep_max = 3
start_time = time.time()

table_data  = []

for olympics in olympic_games.itertuples():
    
    url0 = getattr(olympics, 'url') 
    url  = 'https://en.wikipedia.org/'+url0
    #print(url)
    
    year = getattr(olympics, 'year')
    #print(year)
    
    men_women = getattr(olympics, 'men_women')    
    event = getattr(olympics, 'event')

    page = urllib.request.urlopen(url)
    soup = BeautifulSoup(page, "html.parser")

    time.sleep(np.random.uniform(sleep_min, sleep_max))
                 
    table_MN = pd.read_html(url, match='Rank', attrs={'class': 'wikitable sortable'})
    #print(  len(table_MN), range(len(table_MN)), range(table_MN)  )
    for i in range(len(table_MN)):
        #print(i)
        #print(table_MN[i])

        for j in table_MN[i]:
          table_MN[i]['idx'] = i+1
          table_MN[i]['year'] = year
          table_MN[i]['men_women'] = men_women
          table_MN[i]['event'] = event

        table_data.append(table_MN[i])

df_extract = pd.concat(table_data)
df_extract       

print(len(df_extract))

base_final_trat0 = pd.merge(df_extract, append_final, left_on=['idx', 'year', 'men_women'], right_on=['idx', 'year', 'men_women'], how='inner', indicator=True)
base_final_trat0

# WHEN WE FILTER THE TABLE SCOPE TO CONTAIN ONLY THE COLUMNS YEAR, MEN_WOMEN AND TABLE_NAME, REMOVING THE DUPLICATES, WE GET A TABLE THAT CAN BE
# USED AS A FILTER TO PREVENT CERTAIN TABLES FROM GENERATING BLANK ROWS. FOR EXAMPLE, 1896 AND 1904 THE RESULTS SUMMARY TABLE APPEARS AS IDX5,
# BUT SHOULDN'T BE THERE. AS IN BASE_FINAL_TRAT 1 WE ALREADY HAVE THE TABLE_NAME, IF WE MAKE AN INNER JOIN "FILTER" ONLY THE VALID STEPS (TABLES)
# FOR EACH OF THESE COMPETITIONS

# FILTERING THE COLUMNS 
table_scope_filter = table_scope[['year','men_women','table_name']].drop_duplicates()
table_scope_filter

base_final_trat1 = base_final_trat0[['year'	,'men_women','table_name', 'Rank',	'Athlete'	,'Nation',	'Time',	'Notes'	,'idx'	,	'event']]

print(len(base_final_trat1))
print(len(table_scope_filter))
base_final_trat2 = pd.merge(base_final_trat1, table_scope_filter, left_on=['table_name', 'year', 'men_women'], right_on=['table_name', 'year', 'men_women'], how='inner', indicator=True)
base_final_trat2
print(len(base_final_trat2))

# FINAL TABLE:

base_final_trat2 = base_final_trat2.drop(['_merge'], axis=1)
base_right = pd.merge(base_final_trat2, df_extract, left_on=['idx', 'year', 'men_women'], right_on=['idx', 'year', 'men_women'], how='outer', indicator=True).query('_merge == "right_only"')
base_right.head(5)
